In [1]:
import datasets

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
from datasets import load_dataset
dataset = load_dataset("ai4privacy/pii-masking-300k")

In [27]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 177677
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 47728
    })
})

In [29]:
input_labels = dataset['train'][:29908]['mbert_text_tokens']
target_labels = dataset['train'][:29908]['mbert_bio_labels']
document_ids = dataset['train'][:29908]['id']

In [32]:
train_dataset = [{'id':document_ids[i][:-1], 'ner_tags': target_labels[i], 'tokens': input_labels[i]} for i in range(29908)]

In [33]:
train_dataset[0]

{'id': '40767',
 'ner_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-TIME',
  'I-TIME',
  'I-TIME',
  'O',
  'B-USERNAME',
  'I-USERNAME',
  'O',
  'O',
  'O',
  'B-TIME',
  'I-TIME',
  'I-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'I-USERNAME',
  'O',
  'O',
  'O',
  'O',
  'B-TIME',
  'I-TIME',
  'I-TIME',
  'I-TIME',
  'I-TIME',
  'I-TIME',
  'I-TIME',